In [ ]:
"""
0:      1:      2:      3:      4:
 aaaa    ....    aaaa    aaaa    ....
b    c  .    c  .    c  .    c  b    c
b    c  .    c  .    c  .    c  b    c
 ....    ....    dddd    dddd    dddd
e    f  .    f  e    .  .    f  .    f
e    f  .    f  e    .  .    f  .    f
 gggg    ....    gggg    gggg    ....

  5:      6:      7:      8:      9:
 aaaa    aaaa    aaaa    aaaa    aaaa
b    .  b    .  .    c  b    c  b    c
b    .  b    .  .    c  b    c  b    c
 dddd    dddd    ....    dddd    dddd
.    f  e    f  .    f  e    f  .    f
.    f  e    f  .    f  e    f  .    f
 gggg    gggg    ....    gggg    gggg


a->8# loop 2
b->6# loop 1
c->8# loop 2
d->7# loop 3
e->4# loop 1
f->9# loop 1
g->7# loop 3
"""

In [35]:
"""
https://adventofcode.com/2021/day/8
"""


from typing import NamedTuple, List, Dict
from collections import Counter, defaultdict


RAW1 = """acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf"""
RAW2 = """ be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce"""


class Entry(NamedTuple):
    signal: List[str]
    output: List[str]
        
    @staticmethod
    def parse(line:str)->"Entry":
        split = line.split(" | ")
        signal = ["".join(sorted(wire)) for wire in split[0].split(" ") if wire]
        output = ["".join(sorted(dig)) for dig in split[1].split(" ") if dig]
        return Entry(signal, output)

class Display:
    def __init__(self, text:str):
        self.text = text
        self.entries = self.parse_display()
        self.digits = {
        'abcefg':'0',
        'cf':'1',
        'acdeg':'2',
        'acdfg':'3',
        'bcdf':'4',
        'abdfg':'5',
        'abdefg':'6',
        'acf':'7',
        'abcdefg':'8',
        'abcdfg':'9'
    }
        
    def parse_display(self):
        """
        acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf
        """
        lines = self.text.splitlines()
        entries = [
            Entry.parse(line)
            for line in lines
        ]
        return entries
    
    def count_unique(self):
        len_d = {val: len(k) for k, val in self.digits.items()}
        unique_len = len_d['1'], len_d['4'], len_d['7'], len_d['8']
        counts = []
        for entry in self.entries:
            total = 0
            len_outputs = map(len, entry.output)
            for l in len_outputs:
                if l in unique_len:
                    total += 1
            counts.append(total)
        return sum(counts)
    
    # Part 2
    # I've peeked JG solution for an approach
    # the (crummy) implementation is all mine :) 
    def _create_char_mapping(self, entry:Entry)->Dict['str','str']:
        """
        finds each individual wire characters mapping
        to correct digital display
        """
        chars_map = defaultdict(str)
        chars_left = list('abcdefg')
        counter_wires = Counter(c for wire in entry.signal for c in wire)

        for char, c in counter_wires.items():
            # loop1 freq counts unique of chars in wire
            if c==4:#e
                chars_map['e']=char
                chars_left.remove(char)
            if c==6:#b
                chars_map['b']=char
                chars_left.remove(char)
            if c==9:#f
                chars_map['f']=char
                chars_left.remove(char)
        #print(chars_left)
        for wire in entry.signal:
            # loop 2 get 'c' based on digit 1
            if len(wire)==2: #digit 1 
                list_wire = list(wire)
                #print(list_wire, chars_map['f'])
                char = next(c for c in list_wire if c != chars_map['f'])
                chars_map['c'] = char
                chars_left.remove(char)
                for char, count in counter_wires.items():
                    if count == 8 and char != chars_map['c']:
                        chars_map['a'] = char #a
                        chars_left.remove(char)
                        break

        for wire in entry.signal:
            # loop 3 get 'd' based on digit 4
            if len(wire)==4: # digt 4
                list_wire = list(wire)
                char = next(c # d only char of 4 not captured
                            for c in list_wire 
                            if c not in chars_map.values())
                chars_left.remove(char)
                chars_map['d'] = char
                break

        assert len(chars_left)==1
        
        chars_map['g'] = chars_left[0] # last char left is'g'
        char_map_digits = {val:key for key, val in chars_map.items()}
        return char_map_digits
    
    
    def output_values(self)-> int:
        """
        returns sum of each output entry
        for all entries
        """
        parsed_outputs = []
        for entry in self.entries:
            output_map = self._create_char_mapping(entry)
            digits = []
            for chars in entry.output:
                parsed = "".join(output_map[c] for c in chars)
                digits.append("".join(sorted(parsed)))
            parsed_outputs.append(int("".join(self.digits[d] for d in digits)))
        return sum(parsed_outputs)
        
        

D = Display(text=RAW2)
assert D.count_unique() == 26
assert D.output_values() == 61229

with open('inputs/day8.txt') as f:
    PUZZLE = f.read()
    d=Display(text=PUZZLE)
    print('p1', d.count_unique())
    print('p2', d.output_values())    

p1 449
p2 968175


In [37]:
%timeit d.output_values()

4.17 ms ± 164 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
